# レコメンドデモノートブック

In [ ]:
import json
from pathlib import Path
import random
from concurrent.futures import ThreadPoolExecutor, as_completed


In [ ]:
DATA_PATH = Path('data/candidates.json')
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    candidates = json.load(f)
# スコア降順でソート
candidates = sorted(candidates, key=lambda x: x['score'], reverse=True)
candidates[:3]

In [ ]:
def score_candidate(user, candidate):
    random.seed(hash(f"{user['id']}-{candidate['id']}"))
    return random.uniform(0, 10)


In [ ]:
DEFAULT_THRESHOLD = 7.5
DEFAULT_TOP_K = 3
INJECTION_PROBABILITY = 0.1

def recommend(user, top_k=DEFAULT_TOP_K, threshold=DEFAULT_THRESHOLD):
    cand_list = list(candidates)  # コピー
    low_ranked = cand_list[len(cand_list)//2:]
    for c in low_ranked:
        if random.random() < INJECTION_PROBABILITY:
            idx = random.randint(0, len(cand_list)-1)
            cand_list.insert(idx, c)
    results = []
    with ThreadPoolExecutor() as executor:
        future_map = {executor.submit(score_candidate, user, c): c for c in cand_list}
        for future in as_completed(future_map):
            c = future_map[future]
            score = future.result()
            if score >= threshold:
                results.append({'candidate': c, 'score': score})
            if len(results) >= top_k:
                break
    results.sort(key=lambda x: x['score'], reverse=True)
    return results


In [ ]:
recommend({'id': 1})